#### 视频裁剪增加镜头描述
对视频分片为，首帧和尾帧的描述，总结之后作为训练提示词
注意调整split_video的默认参数，用于处理横屏数据w=540 , h=360或者竖屏数据 w=720 , h=960， 
这个参数按照原视频大小填写


提示词镜头描述使用了glm-4-9b模型，你可以使用在线的平台，也可以单独在本地加载

In [1]:
import cv2
import os
import datetime

### 安装环境后需要重启虚拟机核心

In [2]:
# !pip install Pillow==10.1.0 torch==2.1.2 torchvision==0.16.2 transformers==4.40.0 sentencepiece==0.1.99 accelerate==0.30.1 bitsandbytes==0.43.1 peft==0.9.0

In [3]:
import torch 
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoProcessor

model_path = '/mnt/ceph/develop/jiawei/ComfyUI/models/LLM/MiniCPMv2_6-prompt-generator'
attention = 'sdpa'
precision = 'fp16'
dtype = {"bf16": torch.bfloat16, "fp16": torch.float16, "fp32": torch.float32}[precision]
model = AutoModelForCausalLM.from_pretrained(model_path, attn_implementation=attention,
                                                         torch_dtype=dtype, load_in_4bit=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
 

/mnt/ceph/develop/jiawei/conda_env/keyframe_extra/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-19 15:22:20.695522: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 15:22:21.469016: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 15:22:21.559133: W tensorflow/stream_executor/pla

In [4]:
import cv2
import pandas as pd
import os
import math
import datetime
import torch
import numpy as np
from torch.nn import functional as F
from typing import Union, List
from diffusers.image_processor import VaeImageProcessor

import PIL.Image
from diffusers.utils import export_to_video


def pad_frame(img, scale):
    _, _, h, w = img.shape
    tmp = max(32, int(32 / scale))
    ph = ((h - 1) // tmp + 1) * tmp
    pw = ((w - 1) // tmp + 1) * tmp
    padding = (0,  pw - w, 0, ph - h)
    return F.pad(img, padding)

def pad_video_of_np(samples, w, h):
    print(f"samples dtype:{samples.dtype}")
    print(f"samples shape:{samples.shape}")
    output = []
    # [f, c, h, w]
    for b in range(samples.shape[0]):
        frame = samples[b : b + 1]
        frame = pad_frame(frame, 1).to(dtype=samples.dtype)
        
        frame = F.interpolate(frame, size=(h, w))
        output.append(frame.squeeze(0)) # (to [f, w, h, c])

    image_np = VaeImageProcessor.pt_to_numpy(torch.stack(output))  # (to [49, 512, 480, 3])
    image_pil = VaeImageProcessor.numpy_to_pil(image_np)
    return image_pil


def split_video(video_path, output_dir, start_time, end_time, features_frame, w=540 , h=360 ,chunk_duration=3):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    fps = video_capture.get(cv2.CAP_PROP_FPS)  # Get the frames per second
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frames in the video
    output_files = []
    # Calculate start and end frames based on time
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    video_capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # Set video position to start frame

    current_frame = start_frame
    chunk_index = 0

    while current_frame <= end_frame:
        chunk_frames = []
        for _ in range(int(chunk_duration * fps)):
            ret, frame = video_capture.read()
            if not ret:
                break  
            frame_rgb = frame[..., ::-1]
            
            # 创建一个新数组，确保 stride 是正的
            frame_rgb = frame_rgb.copy()
            tensor = torch.from_numpy(frame_rgb).float().to("cpu", non_blocking=True).float() / 255.0
            chunk_frames.append(
                tensor.permute(2, 0, 1)
            )  # to [c, h, w,]
            current_frame += 1

        if len(chunk_frames) == 0:
            break
            
        # If the last chunk is shorter than expected, repeat the last frame
        while len(chunk_frames) < int(chunk_duration * fps):
            chunk_frames.append(chunk_frames[-1])
            
        pt_frame = torch.from_numpy(np.stack(chunk_frames))  # to [f, c, h, w]
         
        chunk_images = pad_video_of_np(pt_frame,w,h)
     
        # Save the chunk as a video
        output_file = os.path.join(output_dir, f'chunk_{features_frame}_{chunk_index}.mp4')
        print(output_file)
        save_chunk_as_video(chunk_images, output_file, fps=math.ceil((len(chunk_images) - 1) / 6))
        output_files.append(output_file)
        chunk_index += 1

    video_capture.release()
    return output_files
    
 
    
def save_chunk_as_video(tensor: Union[List[np.ndarray], List[PIL.Image.Image]],video_path, fps: int = 8):
    os.makedirs(os.path.dirname(video_path), exist_ok=True)
    export_to_video(tensor, video_path, fps=fps)
    return video_path

#### 使用gpt重新给这个场景增加描述


In [5]:
#!pip install openai

In [7]:
from PIL import Image
from openai import OpenAI

client = OpenAI(
    api_key="9ba20fe4907877f510c193ecfd810b29.qakNNGbHWdF7dzgs",
    base_url="https://open.bigmodel.cn/api/paas/v4/"
)
# Function to encode the image
def encode_image(image_path):
    import base64
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def caption_chunk_gpt(video_path, features_caption):

    video_capture = cv2.VideoCapture(video_path)
    ret, first_frame = video_capture.read()
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
    ret, last_frame = video_capture.read()

    # Get the frame rate (frames per second)
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    
    # Calculate the duration of the video in seconds
    video_duration = total_frames / fps
    
    video_capture.release()
    
    first_frame_rgb = first_frame[..., ::-1] 
    first_frame_rgb = first_frame_rgb.copy()

    last_frame_rgb = last_frame[..., ::-1] 
    last_frame_rgb = last_frame_rgb.copy()
    
    first_frame_pil_image = Image.fromarray(first_frame_rgb)

    last_frame_pil_image = Image.fromarray(last_frame_rgb)

    prompt = """Follow these steps to create a Midjourney-style long prompt for generating high-quality images: 
            1. The prompt should include rich details, vivid scenes, and composition information, capturing the important elements that make up the scene. 
            2. You can appropriately add some details to enhance the vividness and richness of the content, while ensuring that the long prompt does not exceed 256 tokens,you should only return prompt，itself without any additional information"""

    # Prepare the input for the chat method
    msgs = [{"role": "user", "content": [first_frame_pil_image, prompt]}]
    # Use the chat method
    first_frame_generated_text = model.chat(
        image=[first_frame_pil_image],
        msgs=msgs,
        tokenizer=tokenizer,
        processor=processor,
        max_new_tokens=2048,
        sampling=False,
        num_beams=3
    ) 
    
    # Prepare the input for the chat method
    msgs = [{"role": "user", "content": [last_frame_pil_image, prompt]}]
    # Use the chat method
    last_frame_generated_text = model.chat(
        image=[last_frame_pil_image],
        msgs=msgs,
        tokenizer=tokenizer,
        processor=processor,
        max_new_tokens=2048,
        sampling=False,
        num_beams=3
    ) 
    # Sample dictionary of image captions (this should be generated based on your video analysis)
    image_captions = { 
        "1": first_frame_generated_text,
        video_duration: last_frame_generated_text
    }
    
    # Convert image_captions dictionary into a format suitable for new_captions
    new_captions = "\n".join([f"{time}: '{description}'" for time, description in image_captions.items()])

    caption_summary_prompt = f"""Video description: \"{features_caption}\"\n\n
We extracted several frames from this video and described
each frame using an image understanding model, stored in the dictionary variable ‘image_captions: Dict[str: str]‘.
In ‘image_captions‘, the key is the second at which the image appears in the video, and the value is a detailed description
of the image at that moment. Please describe the content of this video in as much detail as possible, based on the
information provided by ‘image_captions‘, including the objects, scenery, animals, characters, and camera
movements within the video. \n image_captions={new_captions}\n\n
You should output your summary directly, and not mention
variables like ‘image_captions‘ in your response.
Do not include ‘\\n’ and the word ’video’ in your response.
Do not use introductory phrases such as: \"The video presents\", \"The video depicts\", \"This video showcases\",
\"The video captures\" and so on.\n Please start the description with the video content directly, such as \"A man
first sits in a chair, then stands up and walks to the kitchen....\"\n Do not use phrases like: \"as the video
progressed\" and \"Throughout the video\".\n Please describe  the content of the video and the changes that occur, in
chronological order.\n Please keep the description of this video within 100 English words."""

    print(f"caption_summary_prompt:{caption_summary_prompt}")
    # 减少信息，生成速度可以在0.1秒完成
    tools = [
        {
            "type": "web_search",
            "web_search": {
                "enable": False, 
            }
        }
    ]
    response = client.chat.completions.create(
        model="glm-4-plus",
        messages=[ {"role": "user", "content": f"{caption_summary_prompt}"}],
        temperature=0,
        tools=tools,
        max_tokens=2000,
    ) 

    caption_summary_text = response.choices[0].message.content
    print(f"{video_path}\r\ncaption_summary_text:{caption_summary_text}")
    return first_frame_generated_text, last_frame_generated_text, caption_summary_text

#### 删除前后时间场景的误差时间 ， 按照3秒每个片段裁剪, fps是原视频的
获取第一帧和最后一帧画面，使用gpt整理视频提示词


In [26]:


def loadcsv_scene_chunk(scene_csv_path, video_path, videos_output_dir, labels_output_dir):
    df = pd.read_csv(scene_csv_path)
    
    # 计算每个分镜编号的总持续时间
    df['持续时间'] = (pd.to_datetime(df['结束时间']) - pd.to_datetime(df['开始时间'])).dt.total_seconds()
    grouped_df = df.groupby('分镜')['持续时间'].sum().reset_index()
    df = df[df['持续时间'] >= 3]
    
    df = df.sort_values(by='持续时间', ascending=True)
        
    # 创建一个新的 DataFrame 来存储行
    new_df = pd.DataFrame(columns=df.columns)
    # 为新数据添加列
    new_df['chunk_index'] = None
    new_df['chunk_path'] = None
    new_df['first_frame_generated_text'] = None
    new_df['last_frame_generated_text'] = None
    new_df['caption_summary_text'] = None
 
    new_df = pd.DataFrame()  # 确保 new_df 已被初始化

    for _, row in df.iterrows(): 
 
        start_time = row['开始时间']
        end_time = row['结束时间']
        features_caption = row['特征描述']
        features_frame = int(row['特征帧']) 
 
        # 将开始时间和结束时间解析为时间对象
        start_time_obj = datetime.datetime.strptime(start_time, "%H:%M:%S,%f")
        end_time_obj = datetime.datetime.strptime(end_time, "%H:%M:%S,%f")
    
        # 计算时间区间的秒数
        start_seconds = (start_time_obj.hour * 3600 + start_time_obj.minute * 60 + start_time_obj.second +
                         start_time_obj.microsecond / 1000000)
        end_seconds = (end_time_obj.hour * 3600 + end_time_obj.minute * 60 + end_time_obj.second +
                       end_time_obj.microsecond / 1000000)
        start_seconds = start_seconds-1
        end_seconds = end_seconds-1
        # Split the video for each start and end time
        output_files = split_video(video_path, videos_output_dir, start_seconds, end_seconds,features_frame)
        print(f"output_files:{output_files}")
        for index, path in enumerate(output_files):
            # 在内层循环中，复制当前行的数据
            new_row = row.copy()
            first_frame_generated_text, last_frame_generated_text, caption_summary_text = caption_chunk_gpt(path, features_caption)
             
            # 更新新行的数据
            new_row['chunk_index'] = f'chunk_{features_frame}_{index}'
            new_row['chunk_path'] = path
            new_row['first_frame_generated_text'] = first_frame_generated_text
            new_row['last_frame_generated_text'] = last_frame_generated_text
            new_row['caption_summary_text'] = caption_summary_text
            
            # 将新行转换为 DataFrame 并追加到 new_df
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
            lable_file = os.path.join(labels_output_dir, f'chunk_{features_frame}_{index}.txt')
            # Open the file in write mode
            with open(lable_file, 'w') as file: 
                file.write(caption_summary_text)

    return new_df

    

In [27]:

def check_output_folder(output_folder):
    if not os.path.isdir(output_folder):
        print('warning: the folder{} is not exist'.format(output_folder))
        # create srt_folder
        os.makedirs(output_folder)
        print('create folder', output_folder)

def split_video_with_chunk(root_path, video_source):
    save_path = f'{root_path}/{video_source}/'
    video_path=f'{save_path}/{video_source}.mp4' 
 
    scene_csv_path=f'{save_path}/scene/{video_source}_scene_keyframe.csv'
    videos_output_dir = f'{save_path}/scene_chunks/videos'
    labels_output_dir = f'{save_path}/scene_chunks/labels'

    scene_chunks_csv_path = f'{save_path}/scene_chunks/{video_source}_scene_chunk_frame.csv'
    check_output_folder(videos_output_dir)

    check_output_folder(labels_output_dir)
    
    new_df = loadcsv_scene_chunk(scene_csv_path,video_path,videos_output_dir, labels_output_dir)
    
    new_df.to_csv(scene_chunks_csv_path, index=False)


In [28]:
split_video_with_chunk('/mnt/ceph/develop/jiawei/lora_dataset/speech_data/test/','test_1')

/tmp/ipykernel_3335373/2572951941.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['持续时间'] = (pd.to_datetime(df['结束时间']) - pd.to_datetime(df['开始时间'])).dt.total_seconds()
/tmp/ipykernel_3335373/2572951941.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['持续时间'] = (pd.to_datetime(df['结束时间']) - pd.to_datetime(df['开始时间'])).dt.total_seconds()


samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (540, 360) to (544, 368) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


/mnt/ceph/develop/jiawei/lora_dataset/speech_data/test//test_1//scene_chunks/videos/chunk_542_0.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[swscaler @ 0x590b180] Warning: data is not aligned! This can lead to a speed loss


...

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (540, 360) to (544, 368) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


/mnt/ceph/develop/jiawei/lora_dataset/speech_data/test//test_1//scene_chunks/videos/chunk_243_1.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[swscaler @ 0x57ba180] Warning: data is not aligned! This can lead to a speed loss


output_files:['/mnt/ceph/develop/jiawei/lora_dataset/speech_data/test//test_1//scene_chunks/videos/chunk_243_0.mp4', '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/test//test_1//scene_chunks/videos/chunk_243_1.mp4']
caption_summary_prompt:Video description: "As the train glides smoothly along the tracks, the sunlight streams through the window, casting a warm glow on the interior. The view outside reveals a serene landscape, with gentle hills stretching into the distance and a clear blue sky overhead. The composition of the scene is balanced, with the train's window framing the natural beauty outside, creating a harmonious blend of man-made and natural elements. The atmosphere is peaceful, evoking a sense of calm and tranquility as the train continues its journey."


We extracted several frames from this video and described
each frame using an image understanding model, stored in the dictionary variable ‘image_captions: Dict[str: str]‘.
In ‘image_captions‘, the key is the second at

/mnt/ceph/develop/jiawei/conda_env/keyframe_extra/lib/python3.10/site-packages/pandas/core/internals/blocks.py:2540: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [29]:


scene_chunks_csv_path = f'/mnt/ceph/develop/jiawei/lora_dataset/speech_data/test/test_1/scene_chunks/test_1_scene_chunk_frame.csv'
df = pd.read_csv(scene_chunks_csv_path)
df

,空白,内容,开始时间,结束时间,分镜,特征帧,特征描述,持续时间,chunk_index,chunk_path,first_frame_generated_text,last_frame_generated_text,caption_summary_text
0,NaN,NaN,"00:00:22,652","00:00:25,695",12,542.0,As the neon lights flicker along the dimly lit...,3.043,chunk_542_0,/mnt/ceph/develop/jiawei/lora_dataset/speech_d...,In the dimly lit interior of a modern art gall...,A long prompt for generating a high-quality im...,"In a dimly lit modern art gallery, two young w..."
1,NaN,NaN,"00:00:22,652","00:00:25,695",12,542.0,As the neon lights flicker along the dimly lit...,3.043,chunk_542_1,/mnt/ceph/develop/jiawei/lora_dataset/speech_d...,"In the dimly lit room, a sense of anticipation...","In the heart of a bustling restaurant, a table...","In a dimly lit room, purple lights cast an eth..."
2,NaN,NaN,"00:00:08,608","00:00:11,739",5,243.0,"As the train glides smoothly along the tracks,...",3.131,chunk_243_0,/mnt/ceph/develop/jiawei/lora_dataset/speech_d...,"In the dimly lit streets of an ancient city, a...","As the train glides smoothly along the tracks,...","In the dimly lit streets of an ancient city, a..."
3,NaN,NaN,"00:00:08,608","00:00:11,739",5,243.0,"As the train glides smoothly along the tracks,...",3.131,chunk_243_1,/mnt/ceph/develop/jiawei/lora_dataset/speech_d...,"As the sun rises, casting a warm glow over the...",In the dimly lit interior of a train compartme...,"As the sun rises, casting a warm glow over the..."


In [29]:
# import os

# root_path = '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/'
# for root, dirs, files in os.walk(root_path):
#     # 如果你只想获取下一层的子目录，可以在这里筛选
#     if root == root_path:
#         # root_dir 下的直接子目录就是 dirs 中的项
#         for dir in dirs:
#             split_video_with_chunk(root_path,dir)
    

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1//scene_chunks/videos/chunk_542.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1//scene_chunks/videos/chunk_542.0_2.mp4
samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1//scene_chunks/videos/chunk_1511.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1//scene_chunks/videos/chunk_1511.0_2.mp4
samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1//scene_chunks/videos/chunk_1511.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([71, 3, 960, 720])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_1//sce

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



samples dtype:torch.float32
samples shape:torch.Size([72, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_11//scene_chunks/videos/chunk_0.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([72, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_11//scene_chunks/videos/chunk_0.0_2.mp4


/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



...

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_7//scene_chunks/videos/chunk_250.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_7//scene_chunks/videos/chunk_250.0_2.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_7//scene_chunks/videos/chunk_250.0_3.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_7//scene_chunks/videos/chunk_18.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_7//scene_chunks/videos/chunk_18.0_2.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_8//scene_chunks/videos/chunk_161.0_1.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_8//scene_chunks/videos/chunk_161.0_2.mp4
samples dtype:torch.float32
samples shape:torch.Size([90, 3, 1920, 1080])
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_8//scene_chunks/videos/chunk_161.0_3.mp4
create folder /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_9//scene_chunks/videos
create folder /mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站//test_9//scene_chunks/labels


/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_1788445/3037519057.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

